# Package Expo: statsmodels

## Installation

In [1]:
#pip install statsmodels

## Linear Regression with Scikit-Learn

### Data Loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [2]:
# insurance dataset
data = pd.read_csv("insurance.csv")
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
# convert categorical variables to indicators; clean up data
data = pd.get_dummies(data).drop(["sex_female", "smoker_no", "region_northeast"], axis=1)
data.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


In [4]:
# set up feature matrix and target array
X = data.drop("charges", axis=1)
y = data[["charges"]]

### Analysis

In [6]:
sk_linear = LinearRegression(fit_intercept=True).fit(X, y)

In [7]:
sk_linear.coef_

array([[  256.85635254,   339.19345361,   475.50054515,  -131.3143594 ,
        23848.53454191,  -352.96389942, -1035.02204939,  -960.0509913 ]])

In [8]:
sk_linear.intercept_

array([-11938.53857617])

In [9]:
sk_yfit = sk_linear.predict(X)
r2_score(y, sk_yfit)

0.7509130345985205

## Linear Regression with statsmodels

### Data Loading

In [10]:
# insurance dataset again
data = pd.read_csv("insurance.csv")
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [11]:
# no cleanup needed
# data = pd.get_dummies(data).drop(["sex_female", "smoker_no", "region_northeast"], axis=1)
# data.head()

### Analysis

In [12]:
# predict charges as a function of other variables
sm_linear = sm.OLS.from_formula("charges ~ age + sex + bmi + children + smoker + region", data=data).fit()

In [13]:
# alternative method, similar to scikit-learn
# X = sm.add_constant(X)
# sm_linear = sm.OLS(y, X).fit()

In [14]:
print(sm_linear.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Mon, 03 May 2021   Prob (F-statistic):               0.00
Time:                        21:20:28   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -1.194e+04    

In [15]:
sm_linear.params

Intercept             -11938.538576
sex[T.male]             -131.314359
smoker[T.yes]          23848.534542
region[T.northwest]     -352.963899
region[T.southeast]    -1035.022049
region[T.southwest]     -960.050991
age                      256.856353
bmi                      339.193454
children                 475.500545
dtype: float64

In [16]:
sm_linear.rsquared

0.7509130345985207

### Aside: Vectorized Operations Example

In [17]:
# assume age is curvilinearly related to charges
sm_linear = sm.OLS.from_formula("charges ~ np.log(age) + sex + bmi + children + smoker + region", data=data).fit()

In [18]:
print(sm_linear.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     483.0
Date:                Mon, 03 May 2021   Prob (F-statistic):               0.00
Time:                        21:20:28   Log-Likelihood:                -13566.
No. Observations:                1338   AIC:                         2.715e+04
Df Residuals:                    1329   BIC:                         2.720e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -3.391e+04   1

## Logistic Regression with statsmodels

<img src="logistic.png">

### Data Loading

In [19]:
# diabetes dataset
data2 = pd.read_csv("diabetes.csv")
data2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Analysis

In [20]:
# predict probability of diabetes (outcome) as function of other variables
sm_logit = sm.Logit.from_formula(
    "Outcome ~ Pregnancies + Glucose + BloodPressure + SkinThickness + Insulin + BMI + DiabetesPedigreeFunction + Age",
    data = data2).fit()

Optimization terminated successfully.
         Current function value: 0.470993
         Iterations 6


In [21]:
print(sm_logit.summary())

                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      759
Method:                           MLE   Df Model:                            8
Date:                Mon, 03 May 2021   Pseudo R-squ.:                  0.2718
Time:                        21:20:28   Log-Likelihood:                -361.72
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 9.652e-54
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -8.4047      0.717    -11.728      0.000      -9.809      -7.000
Pregnancies                  0.1232      0.032      3.840      0.000       0.060       0.

## Saving and Loading Models

In [22]:
from statsmodels.discrete.discrete_model import LogitResults

In [23]:
sm_logit.save("logit_model.pkl")

In [24]:
logit_2 = LogitResults.load("logit_model.pkl")

In [25]:
print(logit_2.summary())

                           Logit Regression Results                           
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      759
Method:                           MLE   Df Model:                            8
Date:                Mon, 03 May 2021   Pseudo R-squ.:                  0.2718
Time:                        21:20:28   Log-Likelihood:                -361.72
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 9.652e-54
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -8.4047      0.717    -11.728      0.000      -9.809      -7.000
Pregnancies                  0.1232      0.032      3.840      0.000       0.060       0.